# 数据提取

- https://eyurtsev.github.io/kor/tutorial.html

In [1]:
!pip install kor langchain

  Using cached langchain-0.0.162-py3-none-any.whl (770 kB)
  Using cached openai-0.27.6-py3-none-any.whl (71 kB)
  Using cached pandas-1.5.3-cp39-cp39-macosx_11_0_arm64.whl (11.0 MB)
  Using cached PyYAML-6.0-cp39-cp39-macosx_11_0_arm64.whl (173 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.7 MB/s eta 0:00:0000:0100:01
  Using cached aiohttp-3.8.4-cp39-cp39-macosx_11_0_arm64.whl (338 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached dataclasses_json-0.5.7-py3-none-any.whl (25 kB)
  Using cached numexpr-2.8.4-cp39-cp39-macosx_11_0_arm64.whl (89 kB)
  Using cached numpy-1.24.3-cp39-cp39-macosx_11_0_arm64.whl (13.9 MB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached pydantic-1.10.7-cp39-cp39-macosx_11_0_arm64.whl (2.6 MB)
  Using cached requests-2.30.0-py3-none-any.whl (62 kB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached attrs-23

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

In [4]:
import json

def json_dump(json_object):
  json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False)
  print(json_formatted_str)

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
)


In [5]:

club_schema = Object(
    id="club",
    description="Information about a club",
    
    attributes=[
        Text(
            id="name",
            description="The name of the club."
        ),
        Text(
            id="manager",
            description="The name of the manager."
        ),
        Number(
            id="ranking",
            description="The ranking of the club."
        )
    ],
    examples=[
        (
            "Napoli managed by Spalletti are the 1st, Juventus managed by Allegri are the 5th.",
            [
                {"name": "Napoli", "manager": "Spalletti", "ranking": 1},
                {"name": "Juventus", "manager": "Allegri", "ranking": 5},
            ],
        )
    ]
)

extraction_chain = create_extraction_chain(llm, club_schema)

In [6]:
text='''
    Manchester City are just 1 point after Arsenal. 
    Arsenal are on top of the table.
    The 2 managers Pep and Arteta are going to meet face to face in this weekend.
'''
output = extraction_chain.predict_and_parse(text=text)['data']

json_dump(output)

{
  "club": [
    {
      "name": "Arsenal",
      "manager": "Arteta",
      "ranking": "1"
    },
    {
      "name": "Manchester City",
      "manager": "Pep",
      "ranking": "2"
    }
  ]
}


In [7]:
stock_market_schema = Object(
    id="stock_market",
    description="A股市场收评，上涨板块综述",
    
    # Notice I put multiple fields to pull out different attributes
    attributes=[
        Text(
            id="industry_name",
            description="行业板块名称"
        )
    ],
    examples=[
        (
            '''
                周二市场呈现出震荡调整格局，盘中行业板块及题材呈现局部轮动和分化，保险、煤炭、通信设备、银行、家用电器、IT设备、有色、电气设备等行业活跃。
                酒店餐饮、半导体、旅游、多元金融、电器仪表、元器件、通用机械、商业连锁、医药等行业呈现回调。
            ''',
            [
                {"industry_name": "保险"},
                {"industry_name": "煤炭"},
                {"industry_name": "通信设备"},
                {"industry_name": "银行"},
                {"industry_name": "家用电器"},
                {"industry_name": "IT设备"},
                {"industry_name": "有色"},
                {"industry_name": "电气设备"}
            ],
        ),
        (
            '''
                截至2023年4月14日收盘，上证指数收涨0.60%，报收3338.15点;深证成指涨0.51%，报收11800.09点;创业板指涨0.93%，报收2428.09点。

                沪深两市全天成交额11150亿元，北向资金净买入64.24亿元，其中沪股通净买入21.58亿元，深股通净买入42.66亿元。

                根据申万二级行业分类，2023年4月14日共有66个行业上涨，其中，能源金属、半导体、电子化学品、工业金属、小金属涨幅居前;

                58个行业下跌，酒店餐饮、旅游及景区、广告营销、互联网电商、数字媒体跌幅居前。

                2023年4月14日，沪深A股共有2655家公司上涨，46家公司涨幅超10%(含)，华星创业、耐科装备、联动科技、拓荆科技、经纬辉开涨幅居前;

                有2102家公司下跌，4家公司跌幅超10%(含)，登康口腔、值得买、奥飞娱乐、慈文传媒、金桥信息跌幅居前。

            ''',
            [
                {"industry_name": "能源金属"},
                {"industry_name": "半导体"},
                {"industry_name": "电子化学品"},
                {"industry_name": "工业金属"},
                {"industry_name": "小金属"}
            ],
        ),
        (
            '''
                传媒、游戏、数据确权、贵金属等板块涨幅居前，工业母机、白酒股等板块飘绿。
            ''',
            [
                {"industry_name": "传媒"},
                {"industry_name": "游戏"},
                {"industry_name": "数字确权"},
                {"industry_name": "贵金属"}
            ],
        )
    ]
)

text='''
    截至2023年4月13日收盘，上证指数收跌0.27%，报收3318.36点;深证成指跌1.21%，报收11739.84点;创业板指跌0.97%，报收2405.76点。

    沪深两市全天成交额11369亿元，北向资金净卖出8688万元，其中沪股通净买入20.64亿元，深股通净卖出21.51亿元。

    根据申万二级行业分类，2023年4月13日共有46个行业上涨，其中，旅游及景区、酒店餐饮、出版、教育、互联网电商涨幅居前;

    78个行业下跌，通信设备、半导体、元件、消费电子、黑色家电跌幅居前。
'''
extraction_chain = create_extraction_chain(llm, stock_market_schema)
output = extraction_chain.predict_and_parse(text=text)['data']

json_dump(output)

{
  "stock_market": [
    {
      "industry_name": "旅游及景区"
    },
    {
      "industry_name": "酒店餐饮"
    },
    {
      "industry_name": "出版"
    },
    {
      "industry_name": "教育"
    },
    {
      "industry_name": "互联网电商"
    }
  ]
}
